In [1]:
import glob

import numpy as np
import pandas as pd

from grafting_classifier import GraftingClassifier
from sklearn.linear_model import SGDClassifier
from ogfs_classifier import OGFSClassifier
from dpp_classifier import DPPClassifier

from sklearn.metrics import log_loss, accuracy_score

#import dask.dataframe as dd
#import dask.array as da

In [2]:
class_train = glob.glob("uci/*_train.csv")
print(class_train)

['uci\\BreastCancer_train.csv', 'uci\\Ionosphere_train.csv', 'uci\\PimaIndiansDiabetes_train.csv']


In [3]:
def train_label(fname):
    targetname = fname.replace(".csv", ".labels")
    return pd.read_csv(targetname)

In [4]:
def get_performance(mod, fpath, base=False):
    train1 = pd.read_csv(fpath).fillna(0)
    y = np.array(train_label(fpath)).flatten()
    
    # simulate streaming...
    # try splitting into groups of ~10,
    # if there is no splits, try ~5.
    train1_cols = np.array_split(range(train1.shape[1]), int(train1.shape[1]/10.0) + 1)
    if len(train1_cols) == 1:
        train1_cols = np.array_split(range(train1.shape[1]), int(train1.shape[1]/5.0) + 1)
    all_cols = []

    #mod = GraftingClassifier(max_iter=5)
    if base:
        mod.fit(train1, y)
        results = {'accuracy': accuracy_score(y, mod.predict(train1)), 
               'logloss': log_loss(y, mod.predict(train1)), 
               'feat_dim': mod.coef_.flatten().shape}
        return results

    for idx, collist in enumerate(train1_cols):
        if idx == 0:
            column_list = list(np.array(list(train1.columns))[collist])
            mod.fit(train1[column_list], y)
            all_cols.extend(list(collist))
        else:
            all_cols.extend(list(collist))
            column_list = list(np.array(list(train1.columns))[all_cols])
            mod.partial_fit(train1[column_list], y)
    
    results = {'accuracy': accuracy_score(y, mod.predict(train1)), 
               'logloss': log_loss(y, mod.predict_proba(train1)), 
               'feat_dim': mod.coef_.flatten().shape}
    return results

In [5]:
models = [
    ('Grafting', GraftingClassifier(max_iter=5)), 
    ('DPP', DPPClassifier(max_iter=5)), 
    ('OGFS', OGFSClassifier(max_iter=5)),
    ('Base', SGDClassifier(max_iter=5))
]

In [6]:
ex_dat = class_train[2]
for nm, mod in models:
    if nm != 'Base':
        print(nm, get_performance(mod, ex_dat))
    else:
        print(nm, get_performance(mod, ex_dat, base=True))

H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\sklearn\linear_model\base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


Grafting {'accuracy': 0.36979166666666669, 'logloss': 21.766624707209335, 'feat_dim': (8,)}


H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2397: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2422: RuntimeWarning: invalid value encountered in double_scalars
  z = (T - mn - correction) / se
H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


DPP {'accuracy': 0.65104166666666663, 'logloss': 12.052593846140708, 'feat_dim': (5,)}
OGFS {'accuracy': 0.38411458333333331, 'logloss': 21.271928691139113, 'feat_dim': (6,)}
Base {'accuracy': 0.40885416666666669, 'logloss': 22.486703120013591, 'feat_dim': (8,)}


In [7]:
ex_dat = class_train[0]
for nm, mod in models:
    if nm != 'Base':
        print(nm, get_performance(mod, ex_dat))
    else:
        print(nm, get_performance(mod, ex_dat, base=True))

Grafting {'accuracy': 0.95422031473533619, 'logloss': 1.2474472544260655, 'feat_dim': (9,)}
DPP {'accuracy': 0.95994277539341921, 'logloss': 1.1202943329221202, 'feat_dim': (6,)}


H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2397: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2422: RuntimeWarning: invalid value encountered in double_scalars
  z = (T - mn - correction) / se
H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


OGFS {'accuracy': 0.97138769670958514, 'logloss': 0.9042392448450044, 'feat_dim': (7,)}
Base {'accuracy': 0.97138769670958514, 'logloss': 22.631081265367843, 'feat_dim': (9,)}


In [8]:
ex_dat = class_train[1]
for nm, mod in models:
    if nm != 'Base':
        print(nm, get_performance(mod, ex_dat))
    else:
        print(nm, get_performance(mod, ex_dat, base=True))

Grafting {'accuracy': 0.85470085470085466, 'logloss': 0.88768970547610793, 'feat_dim': (31,)}


H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2397: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2422: RuntimeWarning: invalid value encountered in double_scalars
  z = (T - mn - correction) / se
H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
H:\Continuum\Anaconda2\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


DPP {'accuracy': 0.88603988603988604, 'logloss': 0.68372044186180336, 'feat_dim': (8,)}
OGFS {'accuracy': 0.58404558404558404, 'logloss': 3.9822166007087429, 'feat_dim': (26,)}
Base {'accuracy': 0.90598290598290598, 'logloss': 12.398822151096725, 'feat_dim': (34,)}
